# Kaggle project

Analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species

In [176]:
import numpy as np
import pandas as pd

In [177]:
# load datasets

weather = pd.read_csv('assets/weather.csv')
spray = pd.read_csv('assets/spray.csv')
train = pd.read_csv('assets/train.csv')
test = pd.read_csv('assets/test.csv')

In train and test data:
- NumMosquitos: number of mosquitoes caught in this trap
- WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 


## Join weather and spray info to a new training dataset

In [178]:
train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [179]:
train.shape

(10506, 12)

In [180]:
def r(x):
    return round(x * 1000)

sl = zip(spray.Latitude.map(r), spray.Longitude.map(r))
rl = zip(train.Latitude.map(r), train.Longitude.map(r))

train['ll_loc'] = zip(train.Latitude.map(r), train.Longitude.map(r))

def check(x):
    if x in sl:
        return False
    else:
        return True

train = train[train.ll_loc.map(check)]

In [181]:
# Create a column which identifies which weather station is closest to the trap

#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

station_1_lat = 41.995
station_1_long = -87.933
station_2_lat = 41.786
station_2_long = -87.752

# Combine lat & long into the one column

train['lat_long_comb'] = zip(train['Latitude'],train['Longitude'])

# Function to calculate distance from trap to each of the weather stations and then which is closest

def station(value):
    lat, lon = value
    dist_1 = np.sqrt(abs(station_1_lat - lat) + abs(station_1_long - lon))
    dist_2 = np.sqrt(abs(station_2_lat - lat) + abs(station_2_long - lon))
    if dist_1 > dist_2:
        return 2
    else:
        return 1

train['closest_station'] = train['lat_long_comb'].apply(station)

In [182]:
train.shape

(9543, 15)

In [183]:
cols= ['Station', 'Date', 'Tavg', 'DewPoint', 'WetBulb', 
       'Heat', 'Cool', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'AvgSpeed']
weather_chosen = weather[cols]

In [184]:
def checkifM(x):
    if x == 'M':
        return np.nan
    else:
        return x
        
weather_chosen = weather_chosen.applymap(checkifM)
weather_chosen.isnull().sum()

Station         0
Date            0
Tavg           11
DewPoint        0
WetBulb         4
Heat           11
Cool           11
StnPressure     4
SeaLevel        9
ResultSpeed     0
AvgSpeed        3
dtype: int64

In [185]:
weather_chosen = weather_chosen.fillna(method='pad')

In [236]:
col = [c for c in weather_chosen.columns.tolist() if c not in ['Date']]
weather_chosen[col] = weather_chosen[col].astype(float)

In [257]:

weather_chosen['Date'] = pd.to_datetime(weather_chosen['Date'])
weather_chosen.set_index('Date', inplace = True)
weather_chosen['Date'] = weather_chosen.index

In [288]:
w_1 = weather_chosen[weather_chosen.Station == 1]
w_2 = weather_chosen[weather_chosen.Station == 2]

In [289]:
t = train.copy()
t.drop(['Address', 'AddressNumberAndStreet', 'll_loc', 'lat_long_comb', 'Street'], axis=1, inplace=True)

In [290]:
from datetime import timedelta

def rolling(value, col):
    date_index = w_1.index.get_loc(value)
    if date_index > 7:
        week_slice = w_1.loc[value - timedelta(days=6):value,col]
        week_slice_df = pd.DataFrame(week_slice, columns = ['Date',col])
        avg = np.mean(week_slice_df[col])
        return avg

In [291]:
for colu in w_1.columns:
    newColumn = 'Rolling' + colu
    w_1[newColumn] = w_1['Date'].apply(rolling, col=colu)

for colu in w_2.columns:
    newColumn = 'Rolling' + colu
    w_2[newColumn] = w_2['Date'].apply(rolling, col=colu)

/Users/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [326]:
# left join
pd.set_option('display.max_columns', 500)

left = t[t.closest_station == 1]
left['Date'] = pd.to_datetime(left['Date'])

print w_1.shape
#pd.merge(to, w_1, how='left', on='Date')
right = w_1.iloc[8:, 10:-1]

(1472, 22)


/Users/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [327]:
pd.merge(left, right, how='left', on='Date')

,Date,Species,Block,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,closest_station,RollingStation,RollingTavg,RollingDewPoint,RollingWetBulb,RollingHeat,RollingCool,RollingStnPressure,RollingSeaLevel,RollingResultSpeed,RollingAvgSpeed
0,2007-05-29,CULEX PIPIENS/RESTUANS,41,T002,41.954690,-87.800991,9,1,0,1,1.0,68.000000,50.571429,58.571429,1.428571,4.428571,29.404286,30.125714,7.642857,9.485714
1,2007-05-29,CULEX RESTUANS,41,T002,41.954690,-87.800991,9,1,0,1,1.0,68.000000,50.571429,58.571429,1.428571,4.428571,29.404286,30.125714,7.642857,9.485714
2,2007-05-29,CULEX RESTUANS,62,T007,41.994991,-87.769279,9,1,0,1,1.0,68.000000,50.571429,58.571429,1.428571,4.428571,29.404286,30.125714,7.642857,9.485714
3,2007-05-29,CULEX PIPIENS/RESTUANS,75,T148,42.017430,-87.687769,8,1,0,1,1.0,68.000000,50.571429,58.571429,1.428571,4.428571,29.404286,30.125714,7.642857,9.485714
4,2007-06-05,CULEX PIPIENS/RESTUANS,41,T002,41.954690,-87.800991,9,3,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571
5,2007-06-05,CULEX RESTUANS,41,T002,41.954690,-87.800991,9,5,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571
6,2007-06-05,CULEX PIPIENS,41,T002,41.954690,-87.800991,9,1,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571
7,2007-06-05,CULEX RESTUANS,91,T009,41.981964,-87.812827,8,2,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571
8,2007-06-05,CULEX RESTUANS,36,T011,41.944869,-87.832763,8,1,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571
9,2007-06-05,CULEX RESTUANS,58,T028,41.986921,-87.689778,9,2,0,1,1.0,69.571429,58.714286,62.428571,1.285714,5.857143,29.078571,29.798571,6.928571,8.328571


## EDA

## Build model

## Get result